# 08 Major Transit QA/QC

In [1]:
import os
import sys
import yaml
import pickle
import glob
import copy

import pandas as pd

from network_wrangler import RoadwayNetwork
from network_wrangler import TransitNetwork
from network_wrangler import ProjectCard
from network_wrangler import Scenario
from network_wrangler import WranglerLogger

from lasso import ModelRoadwayNetwork
from lasso import StandardTransit
from lasso import Parameters
from lasso import mtc

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import logging
logger = logging.getLogger("WranglerLogger")
logger.handlers[0].stream = sys.stdout
logger.setLevel(logging.INFO)

## Remote I/O and Parameters

In [4]:
# root_dir = os.path.join('/Users', 'wsp', 'Documents', 'GitHub', 'travel-model-two-networks')
root_dir = "D:/github/lfs-cleaning/travel-model-two-networks"
input_dir = os.path.join(root_dir, 'data', 'processed', 'version_06')
output_dir = os.path.join(root_dir, 'data', 'processed', 'version_07')
card_dir = os.path.join(root_dir, 'project_cards')
# lasso_dir = os.path.join('/Users', 'wsp', 'Documents', 'GitHub', 'mtc-Lasso')
lasso_dir = "Z:/Data/Users/Sijia/MTC/github/Lasso"

In [5]:
parameters = Parameters(lasso_base_dir = lasso_dir)

2021-03-23 09:49:16, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-03-23 09:49:16, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-03-23 09:49:16, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-03-23 09:49:16, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso


## Read Version 07 Scenario

In [6]:
version_07_pickle_file_name = os.path.join(input_dir, 'working_scenario_07.pickle')
v_07_scenario = pickle.load(open(version_07_pickle_file_name, 'rb'))

In [7]:
v_07_scenario.road_net.links_df.shape

(2190435, 44)

In [11]:
v_07_scenario.transit_net.feed.trips[
    (v_07_scenario.transit_net.feed.trips.route_id == "304")
][["direction_id", "shape_id"]]

,direction_id,shape_id
815,1.0,814
816,1.0,814
817,1.0,814
818,1.0,811
819,1.0,811
824,0.0,808
825,0.0,808
826,0.0,808
827,0.0,805
828,0.0,803


In [12]:
v_07_scenario.transit_net.feed.shapes.shape_id.nunique()

1555

## Create Scenario and Apply Project Cards

In [14]:
# run only once

#transit_net = v_07_scenario.transit_net

#%%time
#transit_net.set_roadnet(v_07_scenario.road_net)

#pickle.dump(transit_net, open(os.path.join(output_dir, 'transit_net.pickle'), 'wb'))

In [15]:
transit_net = pickle.load(open(os.path.join(output_dir, 'transit_net.pickle'), 
                               'rb'))

In [16]:
working_scenario = Scenario.create_scenario(
    base_scenario = {"road_net": v_07_scenario.road_net, 
                     "transit_net": transit_net}
)

2021-03-23 09:52:16, INFO: Creating Scenario
2021-03-23 09:52:16, INFO: Creating Scenario


In [17]:
working_scenario.transit_net.feed.stops.shape

(21757, 18)

In [18]:
v_08_scenario = Scenario.create_scenario(
    base_scenario= working_scenario,
    card_directory = card_dir,
    tags = ["Minor Transit"],
    validate_project_cards=False)

2021-03-23 09:55:43, INFO: Creating Scenario
2021-03-23 09:55:43, INFO: Creating Scenario


z:\data\users\sijia\mtc\github\network_wrangler\network_wrangler\projectcard.py:115: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  attribute_dictionary = yaml.load(_yaml)


In [19]:
len(v_08_scenario.project_cards)

25

In [20]:
v_08_scenario.project_cards[0].__dict__

{'project': 'AC Transit Route 06 Edits',
 'tags': ['Minor Transit'],
 'dependencies': '',
 'changes': [{'category': 'Transit Service Property Change',
   'facility': {'route_id': '6', 'direction_id': 0},
   'properties': [{'property': 'routing',
     'existing': [2544617,
      -2578409,
      -2505212,
      -2539011,
      2520785,
      -2539011,
      -2505212,
      -2578409,
      -2565707,
      -2529318,
      2588825,
      -2537229,
      -2586909,
      -2542550,
      -2533550,
      -2531882,
      -2533765,
      -2563242,
      -2523779,
      2510156,
      -2608258,
      -2538640,
      2595415,
      -2527487,
      2554333,
      -2583896,
      -2537406,
      -2598146,
      -2516414,
      -2607118,
      -2532582,
      2606656,
      -2529117,
      2551953,
      -2592196,
      2528680,
      -2592196,
      -2592471,
      -2533591,
      2536230,
      2572805,
      -2524473,
      -2598689,
      -2529432,
      -2523540,
      -2570811,
      -2589214,
 

In [21]:
v_08_scenario.apply_all_projects()

2021-03-23 09:59:22, INFO: Applying VTA Route 607 Edits
2021-03-23 09:59:22, INFO: Applying VTA Route 607 Edits
2021-03-23 09:59:22, INFO: Applying VTA Route 607 Edits
2021-03-23 09:59:22, INFO: Applying VTA Route 607 Edits
2021-03-23 09:59:22, INFO: Applying Project to Transit Network: VTA Route 607 Edits
2021-03-23 09:59:22, INFO: Applying Project to Transit Network: VTA Route 607 Edits
2021-03-23 09:59:22, INFO: Creating a new stop in stops.txt for node ID: 2031200
2021-03-23 09:59:22, INFO: Creating a new stop in stops.txt for node ID: 2031200
2021-03-23 09:59:22, INFO: Creating a new stop in stops.txt for node ID: 2165214
2021-03-23 09:59:22, INFO: Creating a new stop in stops.txt for node ID: 2165214
2021-03-23 09:59:22, INFO: Creating a new stop in stops.txt for node ID: 2131220
2021-03-23 09:59:22, INFO: Creating a new stop in stops.txt for node ID: 2131220
2021-03-23 09:59:22, INFO: Creating a new stop in stops.txt for node ID: 2043788
2021-03-23 09:59:22, INFO: Creating a new

c:\users\wangs1\.conda\envs\lasso_mtc\lib\site-packages\numpy\core\_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


2021-03-23 09:59:22, INFO: Creating a new stop in stops.txt for node ID: 2146572
2021-03-23 09:59:22, INFO: Creating a new stop in stops.txt for node ID: 2129877
2021-03-23 09:59:22, INFO: Creating a new stop in stops.txt for node ID: 2129877
2021-03-23 09:59:22, INFO: Applying VTA Route 607 Edits
2021-03-23 09:59:22, INFO: Applying VTA Route 607 Edits
2021-03-23 09:59:22, INFO: Applying Project to Transit Network: VTA Route 607 Edits
2021-03-23 09:59:22, INFO: Applying Project to Transit Network: VTA Route 607 Edits
2021-03-23 09:59:22, INFO: Creating a new stop in stops.txt for node ID: 2009535
2021-03-23 09:59:22, INFO: Creating a new stop in stops.txt for node ID: 2009535
2021-03-23 09:59:22, INFO: Creating a new stop in stops.txt for node ID: 2150339
2021-03-23 09:59:22, INFO: Creating a new stop in stops.txt for node ID: 2150339
2021-03-23 09:59:23, INFO: Applying VTA Route 607 Edits
2021-03-23 09:59:23, INFO: Applying VTA Route 607 Edits
2021-03-23 09:59:23, INFO: Applying Proje

2021-03-23 09:59:26, INFO: Creating a new stop in stops.txt for node ID: 2067289
2021-03-23 09:59:26, INFO: Creating a new stop in stops.txt for node ID: 2018674
2021-03-23 09:59:26, INFO: Creating a new stop in stops.txt for node ID: 2018674
2021-03-23 09:59:27, INFO: Applying VTA Route 606 Edits
2021-03-23 09:59:27, INFO: Applying VTA Route 606 Edits
2021-03-23 09:59:27, INFO: Applying Project to Transit Network: VTA Route 606 Edits
2021-03-23 09:59:27, INFO: Applying Project to Transit Network: VTA Route 606 Edits
2021-03-23 09:59:27, INFO: Creating a new stop in stops.txt for node ID: 2064421
2021-03-23 09:59:27, INFO: Creating a new stop in stops.txt for node ID: 2064421
2021-03-23 09:59:27, INFO: Creating a new stop in stops.txt for node ID: 2042744
2021-03-23 09:59:27, INFO: Creating a new stop in stops.txt for node ID: 2042744
2021-03-23 09:59:27, INFO: Creating a new stop in stops.txt for node ID: 2003420
2021-03-23 09:59:27, INFO: Creating a new stop in stops.txt for node ID:

2021-03-23 09:59:30, INFO: Creating a new stop in stops.txt for node ID: 2008138
2021-03-23 09:59:30, INFO: Creating a new stop in stops.txt for node ID: 2026597
2021-03-23 09:59:30, INFO: Creating a new stop in stops.txt for node ID: 2026597
2021-03-23 09:59:30, INFO: Creating a new stop in stops.txt for node ID: 2059777
2021-03-23 09:59:30, INFO: Creating a new stop in stops.txt for node ID: 2059777
2021-03-23 09:59:30, INFO: Creating a new stop in stops.txt for node ID: 2099627
2021-03-23 09:59:30, INFO: Creating a new stop in stops.txt for node ID: 2099627
2021-03-23 09:59:30, INFO: Creating a new stop in stops.txt for node ID: 2084925
2021-03-23 09:59:30, INFO: Creating a new stop in stops.txt for node ID: 2084925
2021-03-23 09:59:31, INFO: Applying VTA Route 605 Edits
2021-03-23 09:59:31, INFO: Applying VTA Route 605 Edits
2021-03-23 09:59:31, INFO: Applying Project to Transit Network: VTA Route 605 Edits
2021-03-23 09:59:31, INFO: Applying Project to Transit Network: VTA Route 6

...

2021-03-23 09:59:39, INFO: Creating a new stop in stops.txt for node ID: 2091183
2021-03-23 09:59:39, INFO: Creating a new stop in stops.txt for node ID: 2091183
2021-03-23 09:59:39, INFO: Creating a new stop in stops.txt for node ID: 2110249
2021-03-23 09:59:39, INFO: Creating a new stop in stops.txt for node ID: 2110249
2021-03-23 09:59:39, INFO: Creating a new stop in stops.txt for node ID: 2152738
2021-03-23 09:59:39, INFO: Creating a new stop in stops.txt for node ID: 2152738
2021-03-23 09:59:39, INFO: Creating a new stop in stops.txt for node ID: 2004027
2021-03-23 09:59:39, INFO: Creating a new stop in stops.txt for node ID: 2004027
2021-03-23 09:59:39, INFO: Creating a new stop in stops.txt for node ID: 2033608
2021-03-23 09:59:39, INFO: Creating a new stop in stops.txt for node ID: 2033608
2021-03-23 09:59:40, INFO: Applying VTA Route 570 Edits
2021-03-23 09:59:40, INFO: Applying VTA Route 570 Edits
2021-03-23 09:59:40, INFO: Applying Project to Transit Network: VTA Route 570 

2021-03-23 09:59:44, INFO: Applying SFMTA Route 381 Edits
2021-03-23 09:59:44, INFO: Applying SFMTA Route 381 Edits
2021-03-23 09:59:44, INFO: Applying Project to Transit Network: SFMTA Route 381 Edits
2021-03-23 09:59:44, INFO: Applying Project to Transit Network: SFMTA Route 381 Edits
2021-03-23 09:59:44, INFO: Creating a new stop in stops.txt for node ID: 1002699
2021-03-23 09:59:44, INFO: Creating a new stop in stops.txt for node ID: 1002699
2021-03-23 09:59:44, INFO: Creating a new stop in stops.txt for node ID: 1015670
2021-03-23 09:59:44, INFO: Creating a new stop in stops.txt for node ID: 1015670
2021-03-23 09:59:45, INFO: Applying SFMTA Route 381 Edits
2021-03-23 09:59:45, INFO: Applying SFMTA Route 381 Edits
2021-03-23 09:59:45, INFO: Applying Project to Transit Network: SFMTA Route 381 Edits
2021-03-23 09:59:45, INFO: Applying Project to Transit Network: SFMTA Route 381 Edits
2021-03-23 09:59:45, INFO: Applying SFMTA Route 381 Edits
2021-03-23 09:59:45, INFO: Applying SFMTA 

2021-03-23 09:59:59, INFO: Applying SFMTA Route 335 Edits
2021-03-23 09:59:59, INFO: Applying Project to Transit Network: SFMTA Route 335 Edits
2021-03-23 09:59:59, INFO: Applying Project to Transit Network: SFMTA Route 335 Edits
2021-03-23 10:00:00, INFO: Applying SFMTA Route 329 Edits
2021-03-23 10:00:00, INFO: Applying SFMTA Route 329 Edits
2021-03-23 10:00:00, INFO: Applying SFMTA Route 329 Edits
2021-03-23 10:00:00, INFO: Applying SFMTA Route 329 Edits
2021-03-23 10:00:00, INFO: Applying Project to Transit Network: SFMTA Route 329 Edits
2021-03-23 10:00:00, INFO: Applying Project to Transit Network: SFMTA Route 329 Edits
2021-03-23 10:00:00, INFO: Applying SFMTA Route 329 Edits
2021-03-23 10:00:00, INFO: Applying SFMTA Route 329 Edits
2021-03-23 10:00:00, INFO: Applying Project to Transit Network: SFMTA Route 329 Edits
2021-03-23 10:00:00, INFO: Applying Project to Transit Network: SFMTA Route 329 Edits
2021-03-23 10:00:00, INFO: Applying SFMTA Route 328 Edits
2021-03-23 10:00:00,

2021-03-23 10:00:17, INFO: Applying SamTrans Route 459 Edits
2021-03-23 10:00:17, INFO: Applying SamTrans Route 459 Edits
2021-03-23 10:00:17, INFO: Applying SamTrans Route 459 Edits
2021-03-23 10:00:17, INFO: Applying Project to Transit Network: SamTrans Route 459 Edits
2021-03-23 10:00:17, INFO: Applying Project to Transit Network: SamTrans Route 459 Edits
2021-03-23 10:00:17, INFO: Creating a new stop in stops.txt for node ID: 2036044
2021-03-23 10:00:17, INFO: Creating a new stop in stops.txt for node ID: 2036044
2021-03-23 10:00:17, INFO: Creating a new stop in stops.txt for node ID: 1511493
2021-03-23 10:00:17, INFO: Creating a new stop in stops.txt for node ID: 1511493
2021-03-23 10:00:18, INFO: Creating a new stop in stops.txt for node ID: 1516920
2021-03-23 10:00:18, INFO: Creating a new stop in stops.txt for node ID: 1516920
2021-03-23 10:00:18, INFO: Creating a new stop in stops.txt for node ID: 1525498
2021-03-23 10:00:18, INFO: Creating a new stop in stops.txt for node ID:

2021-03-23 10:00:24, INFO: Creating a new stop in stops.txt for node ID: 3069892
2021-03-23 10:00:24, INFO: Creating a new stop in stops.txt for node ID: 3018955
2021-03-23 10:00:24, INFO: Creating a new stop in stops.txt for node ID: 3018955
2021-03-23 10:00:24, INFO: Creating a new stop in stops.txt for node ID: 3029560
2021-03-23 10:00:24, INFO: Creating a new stop in stops.txt for node ID: 3029560
2021-03-23 10:00:25, INFO: Applying AC Transit Route 99 Edits
2021-03-23 10:00:25, INFO: Applying AC Transit Route 99 Edits
2021-03-23 10:00:25, INFO: Applying Project to Transit Network: AC Transit Route 99 Edits
2021-03-23 10:00:25, INFO: Applying Project to Transit Network: AC Transit Route 99 Edits
2021-03-23 10:00:25, INFO: Creating a new stop in stops.txt for node ID: 3032871
2021-03-23 10:00:25, INFO: Creating a new stop in stops.txt for node ID: 3032871
2021-03-23 10:00:25, INFO: Creating a new stop in stops.txt for node ID: 2517944
2021-03-23 10:00:25, INFO: Creating a new stop i

In [23]:
v_08_scenario.transit_net.feed.shapes.shape_id.nunique()

1555

In [24]:
v_07_scenario.transit_net.feed.shapes[
    v_07_scenario.transit_net.feed.shapes.shape_id.isin(
        ["1169", "1168", "939", "940", "943", "944"]
    )
].groupby(["shape_id"]).count()

,shape_pt_sequence,shape_osm_node_id,shape_shst_node_id,shape_model_node_id
shape_id,,,,
1168,185,185,185,185
1169,211,211,211,211
939,32,32,32,32
940,167,167,167,167
943,162,162,162,162
944,132,132,132,132


In [25]:
v_08_scenario.transit_net.feed.shapes[
    v_08_scenario.transit_net.feed.shapes.shape_id.isin(
        ["1169", "1168", "939", "940", "943", "944"]
    )
].groupby(["shape_id"]).count()

,shape_pt_sequence,shape_osm_node_id,shape_shst_node_id,shape_model_node_id,shape_pt_lat,shape_pt_lon
shape_id,,,,,,
1168,177,108,108,177,0,0
1169,177,65,65,177,0,0
939,26,22,22,26,0,0
940,161,157,157,161,0,0
943,158,154,154,158,0,0
944,130,128,128,130,0,0


In [27]:
v_07_scenario.transit_net.feed.stops.shape

(21757, 18)

In [28]:
v_08_scenario.transit_net.feed.stops.shape

(22073, 18)

In [29]:
v_07_scenario.transit_net.feed.stop_times.shape

(131897, 21)

In [30]:
v_08_scenario.transit_net.feed.stop_times.shape

(131836, 23)

In [31]:
v_08_scenario.transit_net.feed.trips[
    v_08_scenario.transit_net.feed.trips.shape_id == "1169"
].trip_id

1495    23485
1497    23487
1498    23491
1500    23495
1503    23498
Name: trip_id, dtype: object

In [32]:
v_07_scenario.transit_net.feed.stops[
    v_07_scenario.transit_net.feed.stops.stop_id == "12438"]

,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station,stop_timezone,wheelchair_boarding,stop_code,platform_code,position,direction,stop_id,osm_node_id,shst_node_id,model_node_id
6939,Serramonte Blvd & Hwy 1,NaN,37.66917,-122.47803,1,NaN,0.0,NaN,NaN,NaN,332266,NaN,NaN,NaN,12438,65364289,aae55ed03a14b32416ed1fa359b5266e,1528444


In [33]:
v_07_scenario.transit_net.feed.stops[
    v_07_scenario.transit_net.feed.stops.stop_id == "12439"]

,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station,stop_timezone,wheelchair_boarding,stop_code,platform_code,position,direction,stop_id,osm_node_id,shst_node_id,model_node_id
6940,Serramonte Blvd & Hwy 1,NaN,37.66934,-122.478212,1,NaN,0.0,NaN,NaN,NaN,332267,NaN,NaN,NaN,12439,65364289,aae55ed03a14b32416ed1fa359b5266e,1528444


In [34]:
v_07_scenario.transit_net.feed.stop_times[
    v_07_scenario.transit_net.feed.stop_times.trip_id == "23498"]

,arrival_time,departure_time,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled,timepoint,start_service_area_id,end_service_area_id,...,end_service_area_radius,continuous_pickup,continuous_drop_off,pickup_area_id,drop_off_area_id,pickup_service_area_radius,drop_off_service_area_radius,last_stop_on_trip,trip_id,stop_id
46354,0.0,0.0,1,NaN,0.0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23498,12563
46355,240.0,240.0,2,NaN,0.0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23498,12570
46356,300.0,300.0,3,NaN,0.0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23498,12558
46357,480.0,480.0,4,NaN,0.0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23498,12533
46358,600.0,600.0,5,NaN,0.0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23498,12436
46359,600.0,600.0,6,NaN,0.0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23498,12439
46360,660.0,660.0,7,NaN,0.0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23498,12441
46361,720.0,720.0,8,NaN,0.0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23498,12490
46362,780.0,780.0,9,NaN,0.0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23498,12488
46363,780.0,780.0,10,NaN,0.0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23498,12493


In [35]:
v_08_scenario.transit_net.feed.stop_times[
    v_08_scenario.transit_net.feed.stop_times.trip_id == "23498"]

,arrival_time,departure_time,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled,timepoint,start_service_area_id,end_service_area_id,...,continuous_drop_off,pickup_area_id,drop_off_area_id,pickup_service_area_radius,drop_off_service_area_radius,last_stop_on_trip,trip_id,stop_id,stop_distance,stop_is_skipped
130480,0,0,0,NaN,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,23498,12563,NaN,NaN
130481,240,240,1,NaN,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,23498,12570,NaN,NaN
130482,None,None,2,NaN,None,None,NaN,None,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,23498,101523872,None,None
130483,None,None,3,NaN,None,None,NaN,None,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,23498,12532,None,None
130484,None,None,4,NaN,None,None,NaN,None,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,23498,12436,None,None
130485,None,None,5,NaN,None,None,NaN,None,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,23498,12438,None,None
130486,None,None,6,NaN,None,None,NaN,None,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,23498,12440,None,None
130487,None,None,7,NaN,None,None,NaN,None,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,23498,12484,None,None
130488,None,None,8,NaN,None,None,NaN,None,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,23498,12488,None,None
130489,None,None,9,NaN,None,None,NaN,None,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,23498,12492,None,None


In [36]:
v_08_scenario.applied_projects

['VTA Route 607 Edits',
 'VTA Route 606 Edits',
 'VTA Route 605 Edits',
 'VTA Route 593 Edits',
 'VTA Route 570 Edits',
 'VTA Route 569 Edits',
 'SFMTA Route 381 Edits',
 'SFMTA Route 376 Edits',
 'SFMTA Route 349 Edits',
 'SFMTA Route 346 Edits',
 'SFMTA Route 336 Edits',
 'SFMTA Route 335 Edits',
 'SFMTA Route 329 Edits',
 'SFMTA Route 328 Edits',
 'SFMTA Route 321 Edits',
 'SFMTA Route 307 Edits',
 'SFMTA Route 306 Edits',
 'SFMTA Route 305 Edits',
 'SFMTA Route 304 Edits',
 'SamTrans Route 459 Edits',
 'SamTrans Route 393 Edits',
 'AC Transit Route 99 Edits',
 'AC Transit Route 40 Edits',
 'AC Transit Route 38 Edits',
 'AC Transit Route 06 Edits']

## Write to Disk

## As transit line file (for CUBE)

In [63]:
transfer_fare_df = mtc.write_cube_fare_files(
    roadway_network=v_08_scenario.road_net,
    transit_network=v_08_scenario.transit_net,
    parameters=parameters,
    outpath = os.path.join(output_dir)
)

In [37]:
standard_transit_net = StandardTransit.fromTransitNetwork(v_08_scenario.transit_net, parameters = parameters)
mtc.write_as_cube_lin(standard_transit_net, parameters, outpath = os.path.join(output_dir, "transit.lin"))

2021-03-23 10:07:10, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-03-23 10:07:10, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-03-23 10:07:10, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-03-23 10:07:10, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-03-23 10:07:10, INFO: Converting GTFS Standard Properties to MTC's Cube Standard
2021-03-23 10:07:10, INFO: Converting GTFS Standard Properties to MTC's Cube Standard


#### make agency-specific line files for editing

In [40]:
for agency in standard_transit_net.feed.routes.agency_raw_name.unique():
    sub_transit_net = copy.deepcopy(standard_transit_net)
    sub_transit_net.feed.trips = sub_transit_net.feed.trips[sub_transit_net.feed.trips.agency_raw_name == agency]
    mtc.write_as_cube_lin(sub_transit_net, parameters, outpath = os.path.join(output_dir, agency + "_transit.lin"))

2021-03-23 10:16:38, INFO: Converting GTFS Standard Properties to MTC's Cube Standard
2021-03-23 10:16:38, INFO: Converting GTFS Standard Properties to MTC's Cube Standard
2021-03-23 10:16:46, INFO: Converting GTFS Standard Properties to MTC's Cube Standard
2021-03-23 10:16:46, INFO: Converting GTFS Standard Properties to MTC's Cube Standard
2021-03-23 10:16:47, INFO: Converting GTFS Standard Properties to MTC's Cube Standard
2021-03-23 10:16:47, INFO: Converting GTFS Standard Properties to MTC's Cube Standard
2021-03-23 10:17:17, INFO: Converting GTFS Standard Properties to MTC's Cube Standard
2021-03-23 10:17:17, INFO: Converting GTFS Standard Properties to MTC's Cube Standard
2021-03-23 10:17:20, INFO: Converting GTFS Standard Properties to MTC's Cube Standard
2021-03-23 10:17:20, INFO: Converting GTFS Standard Properties to MTC's Cube Standard
2021-03-23 10:17:24, INFO: Converting GTFS Standard Properties to MTC's Cube Standard
2021-03-23 10:17:24, INFO: Converting GTFS Standard Pr

## As pickle

In [41]:
working_scenario_filename = os.path.join(output_dir, 'working_scenario_08.pickle')
pickle.dump(v_08_scenario, open(working_scenario_filename, 'wb'))

## As standard network

In [39]:
v_08_scenario.transit_net.write(path = output_dir, filename = "v07")

2021-03-23 10:16:15, INFO: Writing transit to directory: D:/github/lfs-cleaning/travel-model-two-networks\data\processed\version_07
2021-03-23 10:16:15, INFO: Writing transit to directory: D:/github/lfs-cleaning/travel-model-two-networks\data\processed\version_07
